<a href="https://colab.research.google.com/github/JOONKYUHONG/DNSC6290_RML/blob/main/GPT3_5_Prompt_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Basic setting

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

In [ ]:
%%time
#Everyone needs to save the dataset in their own drive.
drive.mount("/content/gdrive/")

In [ ]:
%%time
complaints = pd.read_csv('/content/gdrive/MyDrive/WF/complaints.csv')

In [ ]:
# Drop rows where 'Consumer complaint narrative' is null
complaints = complaints.dropna(subset=['Consumer complaint narrative'])

In [ ]:
complaints.isnull().sum()

In [ ]:
complaints_revised = complaints[['Company', 'Consumer complaint narrative']]

In [ ]:
#Choose five banks for sample
company_list = ['WELLS FARGO & COMPANY', 'BANK OF AMERICA, NATIONAL ASSOCIATION', 'CAPITAL ONE FINANCIAL CORPORATION', 'JPMORGAN CHASE $ CO.', 'CITIBANK, N.A.']
complaints_revised = complaints_revised[complaints_revised['Company'].isin(company_list)]
complaints_revised.head(10)

In [ ]:
complaints_revised.info()

In [ ]:
import random

# Specify the sample size you want - 50
sample_size = 50

# Extract the 'Consumer complaint narrative' column as a list
narratives = complaints_revised['Consumer complaint narrative'].tolist()

# Check if the number of narratives is greater than the desired sample size
if len(narratives) > sample_size:
    random_samples = random.sample(narratives, sample_size)
else:
    random_samples = narratives

# Create a DataFrame from the sampled narratives
sample = pd.DataFrame({'Consumer complaint narrative': random_samples})

In [ ]:
sample.head(10)

In [ ]:
sample.isnull().sum()

# 0_Setup
#### Load the API key and relevant Python libaries.

In [ ]:
OPENAI_API_KEY = 'sk-******************************'
# each person has to use own api key

In [ ]:
!pip install openai

In [ ]:
import openai
import os
import time

openai.api_key  = OPENAI_API_KEY

#### helper function
we will use OpenAI's `gpt-3.5-turbo` model and the [chat completions endpoint](https://platform.openai.com/docs/guides/chat).

This helper function will make it easier to use prompts and look at the generated outputs:

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [ ]:
# add error, retry
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0, verbose=False):
    messages = [{"role": "user", "content": prompt}]

    time_start = time.time()
    retry_count = 3
    for i in range(0, retry_count):
        while True:
            try:
                response = openai.ChatCompletion.create(
                    model=model,
                    messages=messages,
                    temperature=temperature, # this is the degree of randomness of the model's output
                )
                answer = response['choices'][0]['message']['content'].strip()
                tokens = response.usage.total_tokens


                time_end = time.time()

                if verbose:
                    print('prompt: %s | token: %d | %.1fsec\nanwer : %s'%(prompt, tokens, (time_end - time_start), answer))
                return answer

            except Exception as error:
                print(f"API Error: {error}")
                print(f"Retrying {i+1} time(s) in 4 seconds...")

                if i+1 == retry_count:
                    return prompt, None, None
                time.sleep(4)
                continue

In [ ]:
prompt = 'Please explain what ChatGPT in 20 words'
answer = get_completion(prompt, model="gpt-3.5-turbo")
print(answer)

# 1_Guidelines for Prompting
two prompting principles and their related tactics in order to write effective prompts for large language models.

## Prompting Principles
- **Principle 1: Write clear and specific instructions**  
- **Principle 2: Give the model time to “think”**

### Tactics

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input,  
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`,  

In [ ]:
text = f"""
You should express what you want a model to do by \
providing instructions that are as clear and \
specific as you can possibly make them. \
This will guide the model towards the desired output, \
and reduce the chances of receiving irrelevant \
or incorrect responses. Don't confuse writing a \
clear prompt with writing a short prompt. \
In many cases, longer prompts provide more clarity \
and context for the model, which can lead to \
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

### Principle 2: Give the model time to “think”

#### Tactic 1: Specify the steps required to complete a task

In [ ]:
text = f"""
In a charming village, siblings Jack and Jill set out on \
a quest to fetch water from a hilltop \
well. As they climbed, singing joyfully, misfortune \
struck—Jack tripped on a stone and tumbled \
down the hill, with Jill following suit. \
Though slightly battered, the pair returned home to \
comforting embraces. Despite the mishap, \
their adventurous spirits remained undimmed, and they \
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

## Synthesize Complaints



* The initial exploratory analysis by the GW project team identified three types of complaints:
 * Complaints that are generally well written - DESIRABLE
 * Complaints with very poor grammar and punctuation - PROBLEMATIC
 * Complaints with frivolous, unrelated information - PROBLEMATIC



Based on this, we had this idea: Have the team focus on just the invariance and robustness aspects related to these “problematic” complaints and focus on questions like “Can the right LLM with the right prompt ‘clean up’ these problematic complaints?” In other words, can an LLM correct the poor grammar and punctuation without changing the meaning (invariance) and can an LLM identify the frivolous content of a complaint (robustness)?





Sample Complaints - Lets see how it looks like

In [ ]:
random_samples[:5]

Prompt for Robustness Testing

In [ ]:
for i in range(len(random_samples[0:5])):
    prompt = f"""
    Your task is to examine if customer \
    complaints from banks are well written without any issue.

    Determine if complaints are well-written (considered desirable), \
    exhibit severe grammar or punctuation issues (considered problematic), \
    or contain irrelevant, frivolous information (also considered problematic). \

    Make your response as short as possible.

    Complaints: ```{random_samples[i]}```
    """

    response = get_completion(prompt)
    print(i, response, "\n")

As we can see some complaints have issues related to grammar. Let's see if LLM can change complaints look better.

Prompt for Equal Complaint Synthesis (MFT)

In [ ]:
for i in range(len(random_samples[0:5])):
    prompt = f"""
    Your task is to generate a short synthesized customer \
    complaints from banks.

    Generate synthesized complaints by rearranging words and sentence structures \
    while maintaining the same meaning and intensity as the original \
    complaints in at most 50 words.

    Focus on maintaining equality in the sentiment. \

    Complaints: ```{random_samples[i]}```
    """

    response = get_completion(prompt)
    print(i, response, "\n")

Prompt for Invariance Complaint Synthesis

In [ ]:
for i in range(len(random_samples[0:5])):
    prompt = f"""
    Your task is to generate a short synthesized customer \
    complaints from banks.

    Create synthetic complaints that retain the core message \
    and sentiment of the original complaints but change some words \
    or phrases while ensuring there is no change in the overall meaning \
    in at most 50 words.

    Complaints: ```{random_samples[i]}```
    """

    response = get_completion(prompt)
    print(i, response, "\n")

Prompt for Harshness Modification (Direction):

In [ ]:
for i in range(len(random_samples[0:5])):
    prompt = f"""
    Your task is to generate a short synthesized customer \
    complaints from banks.

    Develop a method to make complaints more harsh or less harsh \
    while keeping the underlying issue intact in at most 50 words\

    Use a range of intensity levels (Less Harsh or More harsh)) to ensure diversity.

    Complaints: ```{random_samples[i]}```
    """

    response = get_completion(prompt)
    print(i, response, "\n")